In [1]:
import numpy as np
import math

distrib = np.loadtxt("E2.fallos.txt")
mu = distrib.mean()
sigma = distrib.var()
x1 = (0, 0.55)
x2 = (1000, 1.65)
slope = (x2[1] - x1[1]) / (x2[0] - x1[0])
b = x1[1] - slope * x1[1]

def ratio(x):
    return slope * x + b

def truncate(number, digits) -> float:
    stepper = 10.0 ** digits
    return math.trunc(stepper * number) / stepper


In [2]:
class Machine():
    def __init__(self, broken=False, working=True):
        self.break_time = self.get_break_time(0)

    def get_break_time(self, now):
        return math.trunc(np.random.normal(mu, sigma) * 100 + now)

    def set_new_break_time(self, now):
        self.break_time = self.get_break_time(now)


class Repairman():
    def __init__(self):
        self.end_repair_time = self.get_repair_time(0)

    def get_repair_time(self, now):
        maq_frac_hora = np.random.exponential(ratio(now / 100)) * 100
        return math.trunc(maq_frac_hora + now)

    def set_repair_time(self, now):
        self.end_repair_time = self.get_repair_time(now)


def simulate():
    working = [Machine() for i in range(10)]
    waiting = [Machine() for i in range(4)]
    repairmen = [Repairman() for i in range(3)]
    repairmen_working  = []
    waiting_repair = []
    repairing = []
    working_time = 0
    for time in range(100000):
        for i, machine in enumerate(working):
            if machine and machine.break_time == time:
                waiting_repair.append(machine)
                try:
                    new_worker =  waiting.pop()
                    new_worker.set_new_break_time(time)
                    working[i] = new_worker
                except IndexError:
                    working[i] = None

        removers = []
        for i, repairman in enumerate(repairmen):
            if waiting_repair:
                repairman.set_repair_time(time)
                repairmen_working.append(repairman)
                repairing.append(waiting_repair.pop())
                removers.append(i)
        repairmen = [rep for i, rep in enumerate(repairmen) if i not in removers]

        removers = []
        for i, repairman in enumerate(repairmen_working):
            if repairman.end_repair_time == time:
                try:
                    waiting.append(waiting_repair.pop())
                except IndexError:
                    repairmen.append(repairman)
                    removers.append(i)
                    continue
        repairmen_working = [rep for i, rep in enumerate(repairmen_working) if i not in removers]

        if None in working and waiting:
            for _ in range(sum(x is None for x in working)):
                try:
                    working.append(waiting.pop())
                    working.remove(None)
                except IndexError:
                    break
        if None not in working:
            working_time += 1
    return working_time

In [8]:
simulate()

280

In [11]:
suma = 0
for _ in range(100):
    suma += simulate()
suma / 100

278.79

De media está funcionando 2.7 horas de cada 1000. Ni de coña, algo está mal.